In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
import string
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings('ignore')

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load the dataset
df = pd.read_csv('IMDB Dataset.csv')  # Assuming the file is in the same directory

# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
print(df.head())
print("\nValue counts for sentiment:")
print(df['sentiment'].value_counts())

# Visualization 1: Sentiment Distribution
plt.figure(figsize=(8, 6))
sns.countplot(x='sentiment', data=df)
plt.title('Distribution of Sentiments in IMDB Reviews')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.savefig('sentiment_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# Text Preprocessing
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize
    words = word_tokenize(text)
    # Remove stopwords and stem
    words = [stemmer.stem(word) for word in words if word not in stop_words and word not in string.punctuation]
    return ' '.join(words)

# Apply preprocessing to the reviews
print("\nPreprocessing reviews...")
df['cleaned_review'] = df['review'].apply(preprocess_text)

# Visualization 2: Word Cloud for Positive Reviews
positive_reviews = ' '.join(df[df['sentiment']=='positive']['cleaned_review'])
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(positive_reviews)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Word Cloud for Positive Reviews')
plt.axis('off')
plt.savefig('wordcloud_positive.png', dpi=300, bbox_inches='tight')
plt.show()

# Visualization 3: Word Cloud for Negative Reviews
negative_reviews = ' '.join(df[df['sentiment']=='negative']['cleaned_review'])
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(negative_reviews)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Word Cloud for Negative Reviews')
plt.axis('off')
plt.savefig('wordcloud_negative.png', dpi=300, bbox_inches='tight')
plt.show()

# Convert sentiment to numerical values (1 for positive, 0 for negative)
df['sentiment_num'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    df['cleaned_review'], df['sentiment_num'], test_size=0.2, random_state=42)

# Feature Extraction using TF-IDF
print("\nPerforming TF-IDF vectorization...")
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Function to evaluate models
def evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Print classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # Plot confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Negative', 'Positive'], 
                yticklabels=['Negative', 'Positive'])
    plt.title(f'Confusion Matrix - {model.__class__.__name__}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(f'confusion_matrix_{model.__class__.__name__}.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Plot ROC curve if probabilities are available
    if y_pred_proba is not None:
        fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
        roc_auc = auc(fpr, tpr)
        
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC Curve - {model.__class__.__name__}')
        plt.legend(loc="lower right")
        plt.savefig(f'roc_curve_{model.__class__.__name__}.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

# Initialize models
models = {
    'Naive Bayes': MultinomialNB(),
    'SVM': SVC(kernel='linear', probability=True, cache_size=1000, max_iter=500, random_state=42),
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

# Train and evaluate traditional ML models
results = {}
for name, model in models.items():
    print(f"\n{'='*50}")
    print(f"Evaluating {name}...")
    results[name] = evaluate_model(model, X_train_tfidf, y_train, X_test_tfidf, y_test)

# Deep Learning Approach with LSTM
print("\nPreparing data for LSTM...")
# Tokenize the text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to have the same length
max_len = 200  # We'll truncate or pad all reviews to this length
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

# Build LSTM model
lstm_model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=max_len),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("\nLSTM Model Summary:")
lstm_model.summary()

# Train LSTM model
print("\nTraining LSTM model...")
history = lstm_model.fit(
    X_train_pad, y_train,
    epochs=5,
    batch_size=64,
    validation_data=(X_test_pad, y_test),
    verbose=1
)

# Evaluate LSTM model
print("\nEvaluating LSTM model...")
y_pred_lstm = (lstm_model.predict(X_test_pad) > 0.5).astype(int)

# Calculate metrics for LSTM
accuracy = accuracy_score(y_test, y_pred_lstm)
precision = precision_score(y_test, y_pred_lstm)
recall = recall_score(y_test, y_pred_lstm)
f1 = f1_score(y_test, y_pred_lstm)

results['LSTM'] = {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1': f1
}

# Print classification report for LSTM
print("\nClassification Report for LSTM:")
print(classification_report(y_test, y_pred_lstm))

# Plot confusion matrix for LSTM
cm = confusion_matrix(y_test, y_pred_lstm)
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Negative', 'Positive'], 
            yticklabels=['Negative', 'Positive'])
plt.title('Confusion Matrix - LSTM')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('confusion_matrix_LSTM.png', dpi=300, bbox_inches='tight')
plt.show()

# Visualization 5: Training and Validation Accuracy/Loss for LSTM
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('lstm_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

# Compare model performances
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values('accuracy', ascending=False)

# Visualization 6: Model Comparison
plt.figure(figsize=(10, 6))
sns.barplot(x=results_df.index, y=results_df['accuracy'])
plt.title('Model Comparison - Accuracy Scores')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.ylim(0.7, 0.9)
plt.xticks(rotation=45)
plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Print final results
print("\nFinal Model Performance Comparison:")
print(results_df)

In [6]:
pip install matplotlib

     ---------------------------------------- 8.1/8.1 MB 32.2 MB/s eta 0:00:00
     ---------------------------------------- 221.2/221.2 kB ? eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 56.7 MB/s eta 0:00:00
     ---------------------------------------- 71.9/71.9 kB 3.9 MB/s eta 0:00:00
     -------------------------------------- 111.1/111.1 kB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 34.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install seaborn


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/294.9 kB ? eta -:--:--
   ----------------------------- ---------- 215.0/294.9 kB 4.4 MB/s eta 0:00:01
   ---------------------------------------- 294.9/294.9 kB 3.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
python.exe -m pip install --upgrade pip

SyntaxError: invalid syntax (842801469.py, line 1)

In [5]:
!pip install wordcloud


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/299.9 kB ? eta -:--:--
   ------------------------------------ --- 276.5/299.9 kB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 299.9/299.9 kB 4.7 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!pip install nltk


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 57.7/57.7 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.5 MB 15.2 MB/s eta 0:00:01
   ---------------------------------------  1.5/1.5 MB 19.2 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 16.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/274.1 kB ? eta -:--:--
   --------------------------------------- 274.1/274.1 kB 17.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/98.2 kB ? eta -:--:--
   ---------------------------------------- 98.2/98.2 kB 5.9 MB/s eta 0:00:00
   ----------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/49.8 kB ? eta -:--:--
     ---------------------------------------- 49.8/49.8 kB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/375.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/375.9 MB 6.5 MB/s eta 0:00:58
   ---------------------------------------- 1.1/375.9 MB 12.0 MB/s eta 0:00:32
   ---------------------------------------- 3.4/375.9 MB 27.0 MB/s eta 0:00:14
    --------------------------------------- 6.8/375.9 MB 39.4 MB/s eta 0:00:10
    --------------------------------------- 7.8/375.9 MB 35.6 MB/s eta 0:00:11
   - -------------------------------------- 10.8/375.9 MB 46.7 MB/s eta 0:00:08
   - -------------------------------------- 13.5/375.9 MB 54.4 MB/s eta 0:00:07
   - -------------------------------------- 16.4/375.9 MB 54.4 MB/s eta 0:00:07
   -- ------------------------------------- 19.0/375.9 MB 59.8 MB

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import platform
print(platform.architecture())


('64bit', 'WindowsPE')


In [ ]:
pip uninstall tensorflow


In [ ]:
pip install tensorflow-cpu


In [2]:
pip install numpy


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     --------------------------------------- 12.9/12.9 MB 16.0 MB/s eta 0:00:00


In [4]:
pip install pandas

     --------------------------------------- 11.6/11.6 MB 19.8 MB/s eta 0:00:00
     -------------------------------------- 509.2/509.2 kB 3.6 MB/s eta 0:00:00
     ------------------------------------- 347.8/347.8 kB 10.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
pip install saborn

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement saborn (from versions: none)
ERROR: No matching distribution found for saborn

[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!pip install seaborn

     -------------------------------------- 294.9/294.9 kB 4.5 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
pip install wordcloud

     -------------------------------------- 299.8/299.8 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
pip install scikit-learn

     --------------------------------------- 11.1/11.1 MB 43.5 MB/s eta 0:00:00
     ---------------------------------------- 301.8/301.8 kB ? eta 0:00:00
     --------------------------------------- 41.2/41.2 MB 32.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
pip install nltk

     ---------------------------------------- 1.5/1.5 MB 13.6 MB/s eta 0:00:00
     ---------------------------------------- 98.2/98.2 kB 5.5 MB/s eta 0:00:00
     ------------------------------------- 274.0/274.0 kB 17.6 MB/s eta 0:00:00
     ---------------------------------------- 78.5/78.5 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
pip install tensorflow

     -------------------------------------- 375.7/375.7 MB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 3.0/3.0 MB 46.7 MB/s eta 0:00:00
  Using cached opt_einsum-3.4.0-py3-none-any.whl (71 kB)
     ---------------------------------------- 1.5/1.5 MB 47.6 MB/s eta 0:00:00
  Using cached absl_py-2.2.2-py3-none-any.whl (135 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached gast-0.6.0-py3-none-any.whl (21 kB)
  Using cached protobuf-5.29.4-cp310-abi3-win_amd64.whl (434 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl (30 kB)
     ---------------------------------------- 209.7/209.7 kB ? eta 0:00:00
  Using cached tensorboard-2.19.0-py3-none-any.whl (5.5 MB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl (26.4 MB)
     --------------------------------------- 12.9/12.9 MB 32.8 MB/s eta 0:00:00
  Using cached keras-3.9.2-py3-none-any.whl (1.3 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Rohit\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\~umpy.libs\\libscipy_openblas64_-43e11ff0749b8cbe0a615c9cf6737e0e.dll'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
!pip install tensorflow

  Using cached tensorflow-2.19.0-cp310-cp310-win_amd64.whl (375.7 MB)
  Using cached ml_dtypes-0.5.1-cp310-cp310-win_amd64.whl (209 kB)
  Using cached gast-0.6.0-py3-none-any.whl (21 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl (135 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached grpcio-1.71.0-cp310-cp310-win_amd64.whl (4.3 MB)
  Using cached tensorboard-2.19.0-py3-none-any.whl (5.5 MB)
  Using cached h5py-3.13.0-cp310-cp310-win_amd64.whl (3.0 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached keras-3.9.2-py3-none-any.whl (1.3 MB)
  Using cached rich-14.0.0-py3-none-any.whl (243 kB)
  Using cached markdown-3.8-py3-none-any.whl (106 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
